In [4]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
import pandas as pd


In [5]:
!ls ../submits/trash

10epoch.csv	      {i}epoch.csv	  submission (5).csv
5epoch.csv	      n.csv		  submission (6).csv
7epoch.csv	      submission (1).csv  submission (7).csv
9epoch_noregex.csv    submission1.csv	  submission (8).csv
blends		      submission (2).csv  submission.csv
dpcnn_test_preds.csv  submission (3).csv  submission_ridge.csv
flag.csv	      submission (4).csv


In [7]:
sag1 = pd.read_csv('../submits/trash/10epoch.csv')
sag2 = pd.read_csv('../submits/trash/7epoch.csv')
sag3 = pd.read_csv('../submits/trash/{i}epoch.csv')
sag4 = pd.read_csv('../submits/trash/5epoch.csv')
sag5 = pd.read_csv('../submits/trash/9epoch_noregex.csv')
sag6 = pd.read_csv('../submits/trash/blends/average_snip.csv')
sag7 = pd.read_csv('../submits/trash/submission.csv')
sag8 = pd.read_csv('../submits/trash/blends/jigsaw-toxic-comment-classification-challenge/sample_submission.csvblend_it_all (1).csv')
sag9 = pd.read_csv('../submits/trash/flag.csv')

In [8]:
final = (sag1[class_names]*3 + sag2[class_names]*3 + sag3[class_names] + sag4[class_names]*4 + sag5[class_names] + sag6[class_names]*10 + sag7[class_names] + sag8[class_names]*36 + sag9[class_names])/60

In [9]:
25+17

42

In [11]:
final.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.978251,0.349817,0.958285,0.127711,0.905705,0.418895
1,0.023165,0.022486,0.022186,0.021915,0.023202,0.022701
2,0.025993,0.022613,0.024499,0.022093,0.023406,0.022669
3,0.021947,0.022132,0.021888,0.022117,0.022085,0.021923
4,0.028459,0.022227,0.023658,0.021954,0.023649,0.022336


In [12]:
final["id"] = sag1.id

In [13]:
final.to_csv('../submits/trash/blend.csv', index=False)

In [3]:
import pandas as pd
import numpy as np
import os

# Controls weights when combining predictions
# 0: equal average of all inputs; 
# 1: up to 50% of weight going to least correlated input
DENSITY_COEFF = 0.25
assert DENSITY_COEFF >= 0.0 and DENSITY_COEFF <= 1.0

# When merging 2 files with corr > OVER_CORR_CUTOFF 
# the result's weight is the max instead of the sum of the merged files' weights
OVER_CORR_CUTOFF = 0.99
assert OVER_CORR_CUTOFF >= 0.0 and OVER_CORR_CUTOFF <= 1.0

INPUT_DIR = '../submits/trash/'

def load_submissions():
    files = os.listdir(INPUT_DIR)
    csv_files = []
    for f in files:
        if f.endswith(".csv"):
            csv_files.append(f)
    frames = {f:pd.read_csv(INPUT_DIR+f).sort_values('id') for f in csv_files}
    return frames


def get_corr_mat(col,frames):
    c = pd.DataFrame()
    for name,df in frames.items():
        c[name] = df[col]
    cor = c.corr()
    for name in cor.columns:
        cor.set_value(name,name,0.0)
    return cor


def highest_corr(mat,frames):
    n_cor = np.array(mat.values)
    corr = np.max(n_cor)
    idx = np.unravel_index(np.argmax(n_cor, axis=None), n_cor.shape)
    f1 = mat.columns[idx[0]]
    f2 = mat.columns[idx[1]]
    return corr,f1,f2


def get_merge_weights(m1,m2,densities):
    d1 = densities[m1]
    d2 = densities[m2]
    d_tot = d1 + d2
    weights1 = 0.5*DENSITY_COEFF + (d1/d_tot)*(1-DENSITY_COEFF)
    weights2 = 0.5*DENSITY_COEFF + (d2/d_tot)*(1-DENSITY_COEFF)
    return weights1, weights2


def ensemble_col(col,frames,densities):
    if len(frames) == 1:
        _, fr = frames.popitem()
        return fr[col]

    mat = get_corr_mat(col,frames)
    corr,merge1,merge2 = highest_corr(mat,frames)
    new_col_name = merge1 + '_' + merge2

    w1,w2 = get_merge_weights(merge1,merge2,densities)
    new_df = pd.DataFrame()
    new_df[col] = (frames[merge1][col]*w1) + (frames[merge2][col]*w2)
    del frames[merge1]
    del frames[merge2]
    frames[new_col_name] = new_df

    if corr >= OVER_CORR_CUTOFF:
        print('\t',merge1,merge2,'  (OVER CORR)')
        densities[new_col_name] = max(densities[merge1],densities[merge2])
    else:
        print('\t',merge1,merge2)
        densities[new_col_name] = densities[merge1] + densities[merge2]

    del densities[merge1]
    del densities[merge2]
    #print(densities)
    return ensemble_col(col,frames,densities)

jigsaw-toxic-comment-classification-challenge/sample_submission.csv
ens_submission = pd.read_csv('../data/sample_submission.csv.zip').sort_values('id')
#print(get_corr_mat('toxic',load_submissions()))

for col in ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]:
    frames = load_submissions()
    print('\n\n',col)
    densities = {k:1.0 for k in frames.keys()}
    ens_submission[col] = ensemble_col(col,frames,densities)

print(ens_submission)    
ens_submission.to_csv('../submits/trash/blends/lazy_ensemble_submission_99_25.csv', index=False)



 toxic
	 n.csv 9epoch_noregex.csv   (OVER CORR)


/media/stas/fastdata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


	 submission (4).csv submission (3).csv   (OVER CORR)
	 5epoch.csv 7epoch.csv
	 n.csv_9epoch_noregex.csv 5epoch.csv_7epoch.csv
	 10epoch.csv n.csv_9epoch_noregex.csv_5epoch.csv_7epoch.csv
	 flag.csv 10epoch.csv_n.csv_9epoch_noregex.csv_5epoch.csv_7epoch.csv
	 submission (5).csv flag.csv_10epoch.csv_n.csv_9epoch_noregex.csv_5epoch.csv_7epoch.csv
	 dpcnn_test_preds.csv submission (5).csv_flag.csv_10epoch.csv_n.csv_9epoch_noregex.csv_5epoch.csv_7epoch.csv
	 submission1.csv dpcnn_test_preds.csv_submission (5).csv_flag.csv_10epoch.csv_n.csv_9epoch_noregex.csv_5epoch.csv_7epoch.csv
	 submission (1).csv submission1.csv_dpcnn_test_preds.csv_submission (5).csv_flag.csv_10epoch.csv_n.csv_9epoch_noregex.csv_5epoch.csv_7epoch.csv
	 submission (2).csv submission.csv
	 submission (1).csv_submission1.csv_dpcnn_test_preds.csv_submission (5).csv_flag.csv_10epoch.csv_n.csv_9epoch_noregex.csv_5epoch.csv_7epoch.csv submission (2).csv_submission.csv
	 submission (4).csv_submission (3).csv submission (1).cs

	 submission (7).csv submission (5).csv_submission (6).csv_submission (4).csv_submission (3).csv_submission (2).csv_submission.csv_submission (1).csv_submission1.csv_flag.csv_{i}epoch.csv_n.csv_9epoch_noregex.csv_10epoch.csv_5epoch.csv_7epoch.csv
	 submission (8).csv submission (7).csv_submission (5).csv_submission (6).csv_submission (4).csv_submission (3).csv_submission (2).csv_submission.csv_submission (1).csv_submission1.csv_flag.csv_{i}epoch.csv_n.csv_9epoch_noregex.csv_10epoch.csv_5epoch.csv_7epoch.csv
	 dpcnn_test_preds.csv submission (8).csv_submission (7).csv_submission (5).csv_submission (6).csv_submission (4).csv_submission (3).csv_submission (2).csv_submission.csv_submission (1).csv_submission1.csv_flag.csv_{i}epoch.csv_n.csv_9epoch_noregex.csv_10epoch.csv_5epoch.csv_7epoch.csv
	 submission_ridge.csv dpcnn_test_preds.csv_submission (8).csv_submission (7).csv_submission (5).csv_submission (6).csv_submission (4).csv_submission (3).csv_submission (2).csv_submission.csv_submissi